In [11]:
import numpy as np
import os as os
import torch
import torch.nn as nn
import cv2
from PIL import Image

######
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [2]:
# Define the model architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x
    
model = CNN()

input_tensor = torch.randn(1, 3, 70, 70)

output = model.forward(input_tensor)
print(np.array(input_tensor).shape)

(1, 3, 70, 70)


In [3]:
def create_images(img_folder, new_folder):
    
    for dir1 in os.listdir(img_folder):
        
        if dir1 != '.DS_Store':
            
            for file in os.listdir(os.path.join(img_folder, dir1)):
                
                new_name = os.path.splitext(file)
                
                
                image_path = os.path.join(img_folder, dir1)
                new_image_path = os.path.join(new_folder, dir1)

                im1 = Image.open('{}/{}'.format(image_path, file))

                rbg_im1 = im1.convert('RGB')


                rbg_im1.save('{}/{}.jpg'.format(new_image_path, new_name[0]))
                
    return 0

create_images('t_or_l_images', 'fixed_images')

0

In [4]:
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        if dir1 != '.DS_Store':
            for file in os.listdir(os.path.join(img_folder, dir1)):
                
                
       
                image_path= os.path.join(img_folder, dir1,  file)
                image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
                image=cv2.resize(image, (70, 70),interpolation = cv2.INTER_AREA)
                image=np.array(image)
                image = image.astype('float32')
                image /= 255 
                img_data_array.append(image)
                class_name.append(dir1)
                
    return img_data_array, class_name# extract the image array and class name
'''  temp  |  lytic   '''

dataset_X, dataset_y = create_dataset('fixed_images')


In [5]:
dataset_X = np.array(dataset_X)

In [14]:
img_folder = 'fixed_images'
img_data_array, class_name = create_dataset(img_folder)

label_dict = {'temperate': 0, 'lytic': 1}
class_labels = [label_dict[label] for label in class_name]

class_labels_one_hot = to_categorical(class_labels)

X_train, X_test, y_train, y_test = train_test_split(img_data_array, class_labels_one_hot, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(70, 70, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(np.array(X_train), np.array(y_train), batch_size=22, epochs=100, verbose=0, validation_split=0.2)

loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test), verbose=1)
print(f"Test accuracy: {accuracy}")

model.save('temperate_lytic_classifier.h5')


4/4 [==============================] - 0s 28ms/step - loss: 4.2840 - accuracy: 0.4783
Test accuracy: 0.47826087474823


In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Model with more layers, batch normalization, and L1/L2 regularization
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(70, 70, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    
    Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    Dense(232, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    Dense(216, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    Dense(200, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    Dense(180, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    Dense(150, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    Dense(125, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    Dense(90, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    Dense(75, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    Dense(50, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    Dense(25, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    
    # Dropout(0.5),
    
    Dense(2, activation='softmax')
    
])

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True)

# Fit the model with data augmentation
model.fit(datagen.flow(np.array(X_train), np.array(y_train), batch_size=22),
          steps_per_epoch=len(X_train) // 22,
          epochs=100,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          callbacks=[early_stopping])

loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test), verbose=1)
print(f"Test accuracy: {accuracy}")

model.save('temperate_lytic_classifier_v2.h5')


Epoch 1/100


2023-04-25 15:27:29.916105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


20/20 [==============================] - 8s 193ms/step - loss: 1.2860 - accuracy: 0.5046 - val_loss: 1.2533 - val_accuracy: 0.5739
Epoch 2/100
20/20 [==============================] - 3s 161ms/step - loss: 1.2356 - accuracy: 0.5115 - val_loss: 1.2024 - val_accuracy: 0.4696
Epoch 3/100
20/20 [==============================] - 3s 158ms/step - loss: 1.1697 - accuracy: 0.5482 - val_loss: 1.1470 - val_accuracy: 0.5304
Epoch 4/100
20/20 [==============================] - 3s 168ms/step - loss: 1.1201 - accuracy: 0.5092 - val_loss: 1.1030 - val_accuracy: 0.4696
Epoch 5/100
20/20 [==============================] - 3s 164ms/step - loss: 1.0706 - accuracy: 0.5092 - val_loss: 1.0673 - val_accuracy: 0.4696
Epoch 6/100
20/20 [==============================] - 3s 169ms/step - loss: 1.0265 - accuracy: 0.5849 - val_loss: 1.0159 - val_accuracy: 0.4696
Epoch 7/100
20/20 [==============================] - 3s 169ms/step - loss: 0.9864 - accuracy: 0.5436 - val_loss: 0.9789 - val_accuracy: 0.4696
Epoch 8/100

In [6]:


shape = dataset_X.shape

print(dataset_X.shape)

dataset_X = np.reshape(dataset_X, (shape[0],shape[3],shape[1],shape[2]))

print(dataset_X.shape)

output = model.forward(torch.tensor(dataset_X))

print()

(573, 70, 70, 3)
(573, 3, 70, 70)



In [10]:
torch.compile(CNN())

OptimizedModule(
  (_orig_mod): CNN(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fc1): Linear(in_features=8192, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [8]:
# DO NOT RUN THIS IF YOU WANT TO KEEP KERNEL ALIVE

import tensorflow as tf
from tensorflow.keras import layers

# Define the model architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

2023-04-25 14:54:21.031918: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 14:54:22.519407: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
